<a href="https://colab.research.google.com/github/gmprovan/CS4705/blob/main/Assignment_2_Learning_Bayesian_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The purpose of this notebook is to gain some practice in learning graphical models. Your goal is to:

1.   load the Breast Cancer (categorical) data set: https://archive.ics.uci.edu/ml/datasets/breast+cancer
2.   keep the last 20% of the data for testing
3.   compare the performance of 3 learned models on the test data: naive Bayes, tree-structured BN (using the Chow-Liu algorithm), and BN

Below I provide some code fragments to assist with this task.

**Marks:**
*   70%: successful learning of 3 models
*   30%: critical discussion of the reasons for any differences in predictive accuracy



In [ ]:
!pip install  pgmpy
!pip install pandas
!pip install numpy

First learn a naive Bayes model

In [ ]:
from pgmpy.models import NaiveBayes
# create the structure manually to create model_struct

from pgmpy.estimators import MaximumLikelihoodEstimator

mle = MaximumLikelihoodEstimator(model=model_struct, data= ****)

Next, learn a tree-structured model

In [ ]:
from pgmpy.estimators import TreeSearch

# learn graph structure
est = TreeSearch(df_data, root_node="A")
dag = est.estimate(estimator_type="chow-liu")

from pgmpy.estimators import BayesianEstimator

# there are many choices of parametrization, here is one example
model = BayesianNetwork(dag.edges())
model.fit(
    data, estimator=BayesianEstimator, prior_type="dirichlet", pseudo_counts=0.1
)
model.get_cpds()

Finally learn a Bayesian network. **First learn the structure, and then the parameters.**

# **Learning Bayesian Networks**

We now want to learn a Bayesian network, given a set of sample data. Learning a Bayesian network can be split into two problems:

 **Structure learning**: Given a set of data samples, estimate a DAG that captures the dependencies between the variables.

  **Parameter learning**: Given a set of data samples and a DAG that captures the dependencies between the variables, estimate the (conditional) probability distributions of the individual variables.


Methods for doing this include:

Structure learning for discrete, fully observed networks:
    
*    Score-based structure estimation (BIC/BDeu/K2 score; exhaustive search, hill climb/tabu search)
*   Constraint-based structure estimation (PC)

Parameter learning for discrete nodes:

*   Maximum Likelihood Estimation
*   Bayesian Estimation
    





**Structure Learning**

You can use MLE or Bayesian estimation methods.

MLE State counts

To make sense of the given data, we can start by counting how often each state of the variable occurs. If the variable is dependent on parents, the counts are done conditionally on the parents states, i.e. for separately for each parent configuration:

**Bayesian Parameter Estimation**


The Bayesian Parameter Estimator starts with already existing prior CPDs, that express our beliefs about the variables before the data was observed. Those "priors" are then updated, using the state counts from the observed data. 

One can think of the priors as consisting in pseudo state counts, that are added to the actual counts before normalization. Unless one wants to encode specific beliefs about the distributions of the variables, one commonly chooses uniform priors, i.e. ones that deem all states equiprobable.

A very simple prior is the so-called K2 prior, which simply adds 1 to the count of every single state. A somewhat more sensible choice of prior is BDeu (Bayesian Dirichlet equivalent uniform prior). For BDeu we need to specify an equivalent sample size N and then the pseudo-counts are the equivalent of having observed N uniform samples of each variable (and each parent configuration). 

**Maximum Likelihood Estimation**


A natural estimate for the CPDs is to simply use the relative frequencies, with which the variable states have occured. 

This approach is Maximum Likelihood Estimation (MLE). According to MLE, we should fill the CPDs in such a way, that $P(\text{data}|\text{model})$ is maximal. This is achieved when using the relative frequencies.  pgmpy supports MLE as follows:

In [ ]:
from pgmpy.estimators import MaximumLikelihoodEstimator
mle = MaximumLikelihoodEstimator(model, data)



mle.estimate_cpd(variable) computes the state counts and divides each cell by the (conditional) sample size. The mle.get_parameters()-method returns a list of CPDs for all variable of the model.

The built-in fit()-method of BayesianModel provides more convenient access to parameter estimators:

In [6]:
# Calibrate all CPDs of `model` using MLE:
model.fit(data, estimator=MaximumLikelihoodEstimator)

# **Structure Learning**




To learn model structure (a DAG) from a data set, there are two broad techniques:

*   score-based structure learning
*   constraint-based structure learning

In this assignment focus on the score-based approach.


# **Score-based Structure Learning**


This approach construes model selection as an optimization task. It has two building blocks:

A scoring function $s_D\colon M \to \mathbb R$ that maps models to a numerical score, based on how well they fit to a given data set $D$.
A search strategy to traverse the search space of possible models $M$ and select a model with optimal score.


**Scoring functions**


Commonly used scores to measure the fit between model and data are Bayesian Dirichlet scores such as BDeu or K2 and the Bayesian Information Criterion (BIC, also called MDL). 

In [ ]:
import pandas as pd
import numpy as np
from pgmpy.estimators import K2Score, BicScore
from pgmpy.models import BayesianModel



k2 = K2Score(data)
bic = BicScore(data)

#GENERATE MODELS HERE
model1 = BayesianModel(...)
model2 = BayesianModel(...)

#you can compare the performance of the different scoring methods

print(k2.score(model1))
print(bic.score(model1))


print(k2.score(model2))
print(bic.score(model2))


**Search strategies**


The search space of DAGs is super-exponential in the number of variables and the above scoring functions allow for local maxima. The first property makes exhaustive search intractable for all but very small networks, the second prohibits efficient local optimization algorithms to always find the optimal structure. Thus, identifiying the ideal structure is often not tractable. Despite these bad news, heuristic search strategies often yields good results.


Heuristic search: HillClimbSearch implements a greedy local search that starts from the DAG start (default: disconnected DAG) and proceeds by iteratively performing single-edge manipulations that maximally increase the score. The search terminates once a local maximum is found.

In [ ]:
from pgmpy.estimators import HillClimbSearch

est = HillClimbSearch(data)
best_model = est.estimate(scoring_method=BicScore(data))
print(best_model.edges())

In [ ]:
from pgmpy.estimators import BayesianEstimator
est = BayesianEstimator(model, data)




The estimated values in the CPDs are now more conservative. 

BayesianEstimator, too, can be used via the fit()-method. 

In [ ]:
import numpy as np
import pandas as pd
from pgmpy.models import BayesianModel
from pgmpy.estimators import BayesianEstimator


##  model = BayesianModel(****)

model.fit(data, estimator=BayesianEstimator, prior_type="BDeu") # default equivalent_sample_size=5
for cpd in model.get_cpds():
    print(cpd)




# **Discussion**

Please critically compare the performance of the 3 different models.